In [3]:
import io
import requests
from bs4 import BeautifulSoup
import pandas as pd
from statsbombpy import sb
from django.utils.safestring import mark_safe
import time



In [4]:
data_link_file = "../Links/reduced_data_by_season_link.txt"

standard_data_link_file = "../Links/Reduced_links_for_data/standard_stat_by_season.txt"
defense_data_link_file = "../Links/Reduced_links_for_data/defensive_stat_by_season.txt"
keepers_data_link_file = "../Links/Reduced_links_for_data/keepers_stat_by_season.txt"
schedule_data_link_file = "../Links/Reduced_links_for_data/schedules_stat_by_season.txt"
playingtime_data_link_file = "../Links/Reduced_links_for_data/playingtime_stat_by_season.txt"
possession_data_link_file = "../Links/Reduced_links_for_data/possession_stat_by_season.txt"
shooting_data_link_file = "../Links/Reduced_links_for_data/shooting_stat_by_season.txt"
passing_data_link_file = "../Links/Reduced_links_for_data/passing_stat_by_season.txt"
sga_data_link_file = "../Links/Reduced_links_for_data/sga_stat_by_season.txt"

In [5]:

# Définissez l'URL de la page d'accueil de la Ligue des champions
website = "https://fbref.com"
url = "https://fbref.com/en/comps/8/history/Champions-League-Seasons"

# Effectuez une requête HTTP pour récupérer la page d'accueil
response = requests.get(url)

def check_access_url(response):
    return response.status_code == 200

def get_source_code(url):
    result =[]
    response = requests.get(url)
    if response.status_code == 429:
        # Attendre le temps indiqué dans l'en-tête Retry-After
        print("prends une pause et reviens dans",response.headers["Retry-After"], "secondes")
        time.sleep(int(response.headers["Retry-After"]))
        result = get_source_code(url)
    
    if check_access_url(response):
        result = BeautifulSoup(response.content, "html.parser")
        return result

def make_comments_code_visible(soup):
    soup = str(soup)
    soup = soup.replace("<!--", "")
    soup = soup.replace("-->", "")
    soup = mark_safe(soup)
    soup = BeautifulSoup(soup, "html.parser")
    result = soup
    return result

def get_table_saisons_link(url):
    response = requests.get(url)
    if check_access_url(response):
        # Créez un objet BeautifulSoup pour représenter la page d'accueil
        soup = BeautifulSoup(response.content, "html.parser")
        # Trouvez la table des campagnes
        table_saisons_ldc = soup.find("table",{"id":"seasons"})
        body_table_saisons_ldc = table_saisons_ldc.find("tbody")
        ths = body_table_saisons_ldc.find_all("th")
        href_values = list()
        for th in ths:
            href = th.find("a")["href"]
            href_values.append(href)
        links = ths.find_all("a")
        href_values = [link["href"] for link in links]

        for tr in body_table_saisons_ldc.find_all("tr"):
            # Extraire les données de chaque tr
            year = tr.find("th")
            cells = tr.find_all("td")
            data_by_season = []
            for cell in cells[:3]:
                data_by_season.append(cell.text)
            
        return href_values
    return 0

def get_table_saisons_ldc(url):
    # Effectuez une requête HTTP pour récupérer la page d'accueil
    response = requests.get(url)
    data =[]
    if check_access_url(response):
        # Créez un objet BeautifulSoup pour représenter la page d'accueil
        soup = BeautifulSoup(response.content, "html.parser")

        # Trouvez la table des campagnes
        table_saisons_ldc = soup.find("table",{"id":"seasons"})
        #print(table_saisons_ldc)

        theads = table_saisons_ldc.find("thead")
        theaders = theads.find_all("th")
        headers = []
        for h in theaders[:5] :
            headers.append(h.text)

        data.append(headers)
        body_table_saisons_ldc = table_saisons_ldc.find("tbody")
        
        for tr in body_table_saisons_ldc.find_all("tr"):
            # Extraire les données de chaque tr
            year = tr.find("th")
            cells = tr.find_all("td")
            data_by_season = []
            data_by_season.append(year.text)
            for cell in cells[:4]:
                data_by_season.append(cell.text)
            data.append(data_by_season)

    return data

def get_url_for_a_season(url, season):
    return url+season

def save_link_in_txt_file(list_data, filename):
  """
  Sauvegarde une liste de données dans un fichier TXT.

  Args:
    list_data: La liste de données à sauvegarder.
    filename: Le nom du fichier TXT.
  """

  with io.open(filename, "w", encoding="utf-8") as f:
    for data in list_data:
      f.write(data + "\n")


# Vérifiez le code de statut de la réponse HTTP    
#filename_for_data_by_season = "data_by_season_link.txt"
#all_links = get_table_saisons_link(url)
#save_link_in_txt_file(all_links, filename_for_data_by_season)



***Récupération des liens menant aux stats d'une édition de ldc à partir du fichier texte contenant les url pour chaque éditions***

In [6]:
def read_data(file):
    """
    Lit les données d'un fichier txt et les renvoie dans un tableau.

    Args:
    fichier: Le chemin du fichier txt à lire.

    Returns:
    Un tableau contenant les données du fichier.
    """
    # Ouvre le fichier en lecture seule
    with open(file, "r") as f:
        content = f.read()
        content = content.split("\n")

    return content


filename_for_standard_stat_by_season = "../Links/Reduced_links_for_data/gca_stat_by_season.txt"
def save_link(filename_for_standard_stat_by_season, key="stats", save=False):
    link_data = read_data(data_link_file)
    link_standard_stat_by_season = []
    idx = 0
    for l in link_data :
        #print(l)
        l_split = l.split("/")
        #print(l_split)
        if idx>0:
            new_link_standard_list = l_split[1:5]
            if new_link_standard_list !=[]:
                #print(new_link_standard_list)
                new_link_standard_list.append(key)
                new_link_standard_list.append(l_split[-1])
                new_link = ""
                for nl in new_link_standard_list:
                    new_link+=("/"+nl)

                #print(new_link)
                
                link_standard_stat_by_season.append(new_link)
            #print(l_split[-1])
        idx+=1
        
        if save:
            save_link_in_txt_file(link_standard_stat_by_season, filename_for_standard_stat_by_season)
    
save_link(sga_data_link_file, "gca", True)

In [13]:

links_data = dict()
links_data["standard"] = read_data(standard_data_link_file)
links_data["defense"] = read_data(defense_data_link_file)
links_data["keepers"] = read_data(keepers_data_link_file)
links_data["playingtime"] = read_data(playingtime_data_link_file)
links_data["schedule"] = read_data(schedule_data_link_file)
links_data["possession"] = read_data(possession_data_link_file)
links_data["shooting"] = read_data(shooting_data_link_file)
links_data["passing"] = read_data(passing_data_link_file)
links_data["gca"] = read_data(sga_data_link_file)

completes_url = dict()
completes_url["standard"] = []
completes_url["defense"] = []
completes_url["keepers"] = []
completes_url["playingtime"] = []
completes_url["schedule"] = []
completes_url["possession"] = []
completes_url["shooting"] = []
completes_url["passing"] = []
completes_url["gca"] = []
def get_complete_urls(links_data, completes_url) :
    for key in links_data.keys():
        for link in links_data[key]:
            if link != "/"+key+"/" and link != "":
                completes_url[key].append(website+link)
    return completes_url

urls = get_complete_urls(links_data=links_data, completes_url=completes_url)
#for urls in completes_url.keys():
    #print(completes_url[urls])
print(urls)

{'standard': ['https://fbref.com/en/comps/8/2022-2023/stats/2022-2023-Champions-League-Stats', 'https://fbref.com/en/comps/8/2021-2022/stats/2021-2022-Champions-League-Stats', 'https://fbref.com/en/comps/8/2020-2021/stats/2020-2021-Champions-League-Stats', 'https://fbref.com/en/comps/8/2019-2020/stats/2019-2020-Champions-League-Stats', 'https://fbref.com/en/comps/8/2018-2019/stats/2018-2019-Champions-League-Stats', 'https://fbref.com/en/comps/8/2017-2018/stats/2017-2018-Champions-League-Stats'], 'defense': ['https://fbref.com/en/comps/8/2022-2023/defense/2022-2023-Champions-League-Stats', 'https://fbref.com/en/comps/8/2021-2022/defense/2021-2022-Champions-League-Stats', 'https://fbref.com/en/comps/8/2020-2021/defense/2020-2021-Champions-League-Stats', 'https://fbref.com/en/comps/8/2019-2020/defense/2019-2020-Champions-League-Stats', 'https://fbref.com/en/comps/8/2018-2019/defense/2018-2019-Champions-League-Stats', 'https://fbref.com/en/comps/8/2017-2018/defense/2017-2018-Champions-Leag

In [8]:


def get_squads_and_players_stat(key_stats, url_set, id_table_squad_stats, id_table_players_stats):
    stats_standard_data = dict()
    stats_standard_data["squads"] = []
    stats_standard_data["players"] = []
    headers_stats_squads_retrieved = False
    headers_stats_players_retrieved = False
    for url_courant in url_set[key_stats]:
        annee_edition = url_courant.split("/")[6].split("-")[1]
        #print(url_courant)
        #if "2023" in url_courant :
        # Créez un objet BeautifulSoup pour représenter la page d'accueil
        soupStats = get_source_code(url_courant)
        if soupStats != []:
            # Trouvez la table des campagnes
            url_code_with_comments = make_comments_code_visible(soupStats)

            table_standard_stat_by_squad = url_code_with_comments.find("table",{"id":id_table_squad_stats})
            table_players_stats_contents = url_code_with_comments.find("table", {"id":id_table_players_stats})
            
            if table_standard_stat_by_squad != None:
                if headers_stats_squads_retrieved == False :
                    headers_stats_squads_retrieved = True
                    theaders_standard_stats_by_squad = table_standard_stat_by_squad.find("thead")
                    headers_standard_stats_by_squad = theaders_standard_stats_by_squad.find_all("tr")
                    headers_squad_standard = []
                    for idx in range(len(headers_standard_stats_by_squad)):
                        if idx == 1:
                            th_headers_standard_stats_by_squad = headers_standard_stats_by_squad[idx].find_all("th")
                            for th in th_headers_standard_stats_by_squad :
                                headers_squad_standard.append(th.text)
                            headers_squad_standard.append("Année édition")
                    stats_standard_data["squads"].append(headers_squad_standard)


                body_table_standard_stat_by_squad = table_standard_stat_by_squad.find("tbody")
                tr_headers_standard_by_squad = body_table_standard_stat_by_squad.find_all("tr")
                for tr in tr_headers_standard_by_squad:
                    row = []
                    a = tr.find("a")
                    row.append(a.text)
                    for td in tr.find_all("td"):
                        row.append(td.text)
                    row.append(annee_edition)
                    stats_standard_data["squads"].append(row)


            if table_players_stats_contents != None :
                theaders_standard_stats_by_players = table_players_stats_contents.find("thead")
                headers_standard_stats_by_players = theaders_standard_stats_by_players.find_all("tr")
                headers_players_standard = []
                for idx in range(len(headers_standard_stats_by_players)):
                    if idx == 1:
                        th_headers_standard_stats_by_players = headers_standard_stats_by_players[idx].find_all("th")
                        for th in th_headers_standard_stats_by_players :
                            headers_players_standard.append(th.text) 
                        headers_players_standard.append("Année édition")
                stats_standard_data["players"].append(headers_players_standard)
                
                body_table_standard_stat_by_players = table_players_stats_contents.find("tbody")
                tr_headers_standard_by_players = body_table_standard_stat_by_players.find_all("tr")
                for tr in tr_headers_standard_by_players:
                    row = []
                    th_row = tr.find("th")
                    a = tr.find("a")
                    if th_row != None:
                        row.append(th_row.text)
                    if a != None :
                        #print(a.text)
                        row.append(a.text)
                    for td in tr.find_all("td"):
                        atd = td.find("a")
                        if atd != None:
                            row.append(atd.text)
                        else:
                            row.append(td.text)
                    row.append(annee_edition)        
                    stats_standard_data["players"].append(row)
    return stats_standard_data     



def save_data_frame_to_csv(data_to_save, file_path):
    data_frame = pd.DataFrame(data=data_to_save[1:])
    #data_frame.dropna()
    #print(data_to_save[0])
    #print(len(data_frame))
    
    #for d in data_to_save:
    #    print(len(d))
    #    print(d)
    data_frame.to_csv(file_path, index=False, header=data_to_save[0])



In [14]:
all_sga_stats = get_squads_and_players_stat("gca", urls, "stats_squads_gca_for", "stats_gca")

**Sauvegarde stats standard**

In [35]:
#all_standard_stats = get_squads_and_players_stat("standard", urls, "stats_squads_standard_for", "stats_standard")

In [106]:
#file_path_standard_stat_squads = "../Datas_project/Reduced_datas/standard_squads_stats.csv"
#file_path_standard_stat_players = "../Datas_project/Reduced_datas/player_stats/standard_players_stats.csv"

#for s in all_standard_stats["squads"]:
#    print(s)

#save_data_frame_to_csv(all_standard_stats["squads"], file_path_standard_stat_squads)
stats_players_by_year = dict()
stats_players_all_year = list()
stats_players_all_year.append(all_standard_stats["players"][0])
for r in all_standard_stats["players"][1:] :
    if r[-1] not in stats_players_by_year.keys():
        stats_players_by_year[r[-1]] = list()
    irow = []
    for i_r in range(len(r)):
        if i_r !=2 and len(r)>8:
            irow.append(r[i_r])

    if len(irow)>8 and "Année édition" and "npxG" not in irow:
        #print(irow)
        stats_players_all_year.append(irow)
    #print(irow)

#save_data_frame_to_csv(stats_players_all_year, file_path_standard_stat_players)
#for d in stats_players_by_year.keys() :
    #for i in range(len(stats_players_by_year[d])):
        #print(stats_players_by_year[d][i])

**Sauvegarde defense stats**

In [97]:
#all_defense_stats = get_squads_and_players_stat("defense", urls, "stats_squads_defense_for", "stats_defense")
"""for url in urls["defense"]:
    checking_url = check_access_url(requests.get(url))
    if checking_url != True:
        print(url)"""

'for url in urls["defense"]:\n    checking_url = check_access_url(requests.get(url))\n    if checking_url != True:\n        print(url)'

In [107]:
#file_path_standard_defense_squads = "../Datas_project/Reduced_datas/defense_squads_stats.csv"
#file_path_standard_defense_players = "../Datas_project/Reduced_datas/player_stats/defense_players_stats.csv"

#save_data_frame_to_csv(all_defense_stats["squads"], file_path_standard_defense_squads)
defense_players_by_year = dict()
defense_players_all_year = list()
defense_players_all_year.append(all_defense_stats["players"][0])
for r in all_defense_stats["players"][1:] :
    if r[-1] not in defense_players_by_year.keys():
        defense_players_by_year[r[-1]] = list()
    irow = []
    for i_r in range(len(r)):
        if i_r !=2 and len(r)>8:
            irow.append(r[i_r])
    if len(irow)>8 and "Tkl%" not in irow:
        #print(irow)
        defense_players_all_year.append(irow)
    #print(irow)

#save_data_frame_to_csv(defense_players_all_year, file_path_standard_defense_players)

**Sauvegarde keeper stats**

In [100]:
#all_keepers_stats = get_squads_and_players_stat("keepers", urls, "stats_squads_keeper_for", "stats_keeper")
"""for url in urls["keepers"]:
    checking_url = check_access_url(requests.get(url))
    if checking_url != True:
        print(url)"""

'for url in urls["keepers"]:\n    checking_url = check_access_url(requests.get(url))\n    if checking_url != True:\n        print(url)'

In [108]:
#file_path_standard_keeper_squads = "../Datas_project/Reduced_datas/keeper_squads_stats.csv"
#file_path_standard_keeper_players = "../Datas_project/Reduced_datas/player_stats/keeper_players_stats.csv"

#save_data_frame_to_csv(all_keepers_stats["squads"], file_path_standard_keeper_squads)
keeper_players_by_year = dict()
keeper_players_all_year = list()
keeper_players_all_year.append(all_keepers_stats["players"][0])
for r in all_keepers_stats["players"][1:] :
    if r[-1] not in keeper_players_by_year.keys():
        keeper_players_by_year[r[-1]] = list()
    irow = []
    for i_r in range(len(r)):
        if i_r !=2 and len(r)>8:
            irow.append(r[i_r])
    if len(irow)>8 and not "Save%" in irow:
        #print(irow)
        keeper_players_all_year.append(irow)
    #print(irow)

#save_data_frame_to_csv(keeper_players_all_year, file_path_standard_keeper_players)

**Sauvagarde stats des matchs**

In [37]:

def get_schedules_stat(key_stats, url_set, id_table_squad_stats):
    stats_schedules_data = dict()
    stats_schedules_data["matchs"] = []

    headers_stats_squads_retrieved = False
    for url_courant in url_set[key_stats]:
        annee_edition = url_courant.split("/")[6].split("-")[1]
        #print(url_courant)
        #if "2023" in url_courant :
        # Créez un objet BeautifulSoup pour représenter la page d'accueil
        soupStats = get_source_code(url_courant)
        if soupStats != []:
            # Trouvez la table des campagnes
            url_code_with_comments = make_comments_code_visible(soupStats)

            table_standard_stat_by_squad = url_code_with_comments.find("table",{"id":id_table_squad_stats})
            
            if table_standard_stat_by_squad != None:
                if headers_stats_squads_retrieved == False :
                    headers_stats_squads_retrieved = True
                    theaders_standard_stats_by_squad = table_standard_stat_by_squad.find("thead")
                    headers_standard_stats_by_squad = theaders_standard_stats_by_squad.find("tr")
                    headers_squad_standard = []
                    for th_header in headers_standard_stats_by_squad.find_all("th"):
                        headers_squad_standard.append(th_header.text)
                    headers_squad_standard.append("Année édition")
                    stats_schedules_data["matchs"].append(headers_squad_standard)


                body_table_standard_stat_by_squad = table_standard_stat_by_squad.find("tbody")
                tr_headers_standard_by_squad = body_table_standard_stat_by_squad.find_all("tr")
                for tr in tr_headers_standard_by_squad:
                    row = []
                    th_round = tr.find("th")
                    if th_round != None:
                        th_a_round = th_round.find("a")
                        if th_a_round != None:
                            row.append(th_a_round.text)
                    for td in tr.find_all("td"):
                        td_a = td.find("a")
                        if td_a != None:
                            row.append(td_a.text)
                        else :
                            row.append(td.text)
                    row.append(annee_edition)
                    stats_schedules_data["matchs"].append(row)


    return stats_schedules_data    



#for url in urls["schedule"]:
    #checking_url = check_access_url(requests.get(url))
    #if checking_url != True:
    #    print(url)

In [38]:
all_schedules_stats = get_schedules_stat("schedule", urls, "sched_all")

In [39]:
schedule_stats = list()
for kk in range(len(all_schedules_stats["matchs"])):
    count = 0
    for i in all_schedules_stats["matchs"][kk]:
        if "" == i:
            count+=1
    if count <8 :
        #print(all_schedules_stats["matchs"][kk])
        schedule_stats.append(all_schedules_stats["matchs"][kk])
file_path_schedule_stat = "../Datas_project/Reduced_datas/schedule_stats.csv"
save_data_frame_to_csv(schedule_stats, file_path_schedule_stat)

In [ ]:
for url in urls["schedule"]:
    checking_url = check_access_url(requests.get(url))
    if checking_url != True:
        print(url)

In [ ]:
#file_path_keeper_stat_squads = "Datas_project/keeper_squads_stats.csv"
#save_data_frame_to_csv(all_keepers_stats["squads"], file_path_keeper_stat_squads)
#for k in all_keepers_stats["squads"]:
    #for kk in all_keepers_stats[k]:
    #print(k)
# Faire une requête au site web
#response2 = requests.get("https://fbref.com/en/comps/8/history/Champions-League-Seasons")

# Obtenir l'en-tête X-RateLimit-Remaining
#rate_limit_remaining = response2.headers

# Afficher le nombre de requêtes restantes
#print(rate_limit_remaining)

**Sauvegarde playing time**

In [ ]:
all_playing_stats = get_squads_and_players_stat("playingtime", urls, "stats_squads_playing_time_for", "stats_playing_time")

In [36]:
file_path_playingtime_squads = "../Datas_project/Reduced_datas/playingtime_squads_stats.csv"
file_path_playingtime_players = "../Datas_project/Reduced_datas/player_stats/playingtime_players_stats.csv"


save_data_frame_to_csv(all_playing_stats["squads"], file_path_playingtime_squads)
playingtime_players_by_year = dict()
playingtime_players_all_year = list()
playingtime_players_all_year.append(all_playing_stats["players"][0])
for r in all_playing_stats["players"][1:] :
    if r[-1] not in playingtime_players_by_year.keys():
        playingtime_players_by_year[r[-1]] = list()
    irow = []
    for i_r in range(len(r)):
        if i_r !=2 and len(r)>8:
            irow.append(r[i_r])
    if len(irow)>8 and "Mn/Sub" not in irow:
        #print(irow)
        playingtime_players_all_year.append(irow)


save_data_frame_to_csv(playingtime_players_all_year, file_path_playingtime_players)

**Acquisition données possession passe shoot**

In [32]:
#print(urls)
all_possession_stats = get_squads_and_players_stat("possession", urls, "stats_squads_possession_for", "stats_possession")
all_shooting_stats = get_squads_and_players_stat("shooting", urls, "stats_squads_shooting_for", "stats_shooting")
all_passing_stats = get_squads_and_players_stat("passing", urls, "stats_squads_passing_for", "stats_passing")
all_sga_stats = get_squads_and_players_stat("passing", urls, "stats_squads_passing_for", "stats_passing")

**Possession**

In [33]:
file_path_possession_squads = "../Datas_project/Reduced_datas/possession_squads_stats.csv"
file_path_possession_players = "../Datas_project/Reduced_datas/player_stats/possession_players_stats.csv"


save_data_frame_to_csv(all_possession_stats["squads"], file_path_possession_squads)
possession_players_by_year = dict()
possession_players_all_year = list()
possession_players_all_year.append(all_possession_stats["players"][0])
for r in all_possession_stats["players"][1:] :
    if r[-1] not in possession_players_by_year.keys():
        possession_players_by_year[r[-1]] = list()
    irow = []
    for i_r in range(len(r)):
        if i_r !=2 and len(r)>8:
            irow.append(r[i_r])
    if len(irow)>8 and "Succ%" not in irow:
        #print(irow)
        possession_players_all_year.append(irow)


save_data_frame_to_csv(possession_players_all_year, file_path_possession_players)

**Shooting**

In [36]:
file_path_shooting_squads = "../Datas_project/Reduced_datas/shooting_squads_stats.csv"
file_path_shooting_players = "../Datas_project/Reduced_datas/player_stats/shooting_players_stats.csv"


save_data_frame_to_csv(all_shooting_stats["squads"], file_path_shooting_squads)
shooting_players_by_year = dict()
shooting_players_all_year = list()
shooting_players_all_year.append(all_shooting_stats["players"][0])
for r in all_shooting_stats["players"][1:] :
    if r[-1] not in shooting_players_by_year.keys():
        shooting_players_by_year[r[-1]] = list()
    irow = []
    for i_r in range(len(r)):
        if i_r !=2 and len(r)>8:
            irow.append(r[i_r])
    if len(irow)>8 and "SoT%" not in irow:
        #print(irow)
        shooting_players_all_year.append(irow)


save_data_frame_to_csv(shooting_players_all_year, file_path_shooting_players)

**Passing**

In [37]:
file_path_passing_squads = "../Datas_project/Reduced_datas/passing_squads_stats.csv"
file_path_passing_players = "../Datas_project/Reduced_datas/player_stats/passing_players_stats.csv"


save_data_frame_to_csv(all_passing_stats["squads"], file_path_passing_squads)
passing_players_by_year = dict()
passing_players_all_year = list()
passing_players_all_year.append(all_passing_stats["players"][0])
for r in all_passing_stats["players"][1:] :
    if r[-1] not in passing_players_by_year.keys():
        passing_players_by_year[r[-1]] = list()
    irow = []
    for i_r in range(len(r)):
        if i_r !=2 and len(r)>8:
            irow.append(r[i_r])
    if len(irow)>8 and "Cmp%" not in irow:
        #print(irow)
        passing_players_all_year.append(irow)


save_data_frame_to_csv(passing_players_all_year, file_path_passing_players)

**Création tirs / but**

In [15]:
print(all_sga_stats)

{'squads': [['Squad', '# Pl', '90s', 'SCA', 'SCA90', 'PassLive', 'PassDead', 'TO', 'Sh', 'Fld', 'Def', 'GCA', 'GCA90', 'PassLive', 'PassDead', 'TO', 'Sh', 'Fld', 'Def', 'Année édition'], ['Ajax', '19', '6.0', '110', '18.33', '93', '8', '2', '4', '2', '1', '20', '3.33', '16', '1', '1', '1', '1', '0', '2023'], ['Atlético Madrid', '22', '6.0', '168', '28.00', '128', '9', '6', '13', '6', '6', '7', '1.17', '6', '0', '0', '1', '0', '0', '2023'], ['Barcelona', '26', '6.0', '166', '27.67', '129', '10', '9', '5', '6', '7', '23', '3.83', '20', '1', '0', '1', '0', '1', '2023'], ['Bayern Munich', '24', '10.0', '302', '30.20', '238', '12', '24', '15', '9', '4', '39', '3.90', '30', '2', '4', '3', '0', '0', '2023'], ['Benfica', '24', '10.0', '254', '25.40', '195', '12', '18', '10', '16', '3', '47', '4.70', '31', '1', '5', '1', '6', '3', '2023'], ['Celtic', '22', '6.0', '141', '23.50', '109', '10', '8', '8', '3', '3', '5', '0.83', '3', '0', '0', '1', '1', '0', '2023'], ['Chelsea', '26', '10.0', '283',

In [16]:
file_path_sga_squads = "../Datas_project/Reduced_datas/sga_squads_stats.csv"
file_path_sga_players = "../Datas_project/Reduced_datas/player_stats/sga_players_stats.csv"


save_data_frame_to_csv(all_sga_stats["squads"], file_path_sga_squads)
sga_players_by_year = dict()
sga_players_all_year = list()
sga_players_all_year.append(all_sga_stats["players"][0])
for r in all_sga_stats["players"][1:] :
    if r[-1] not in sga_players_by_year.keys():
        sga_players_by_year[r[-1]] = list()
    irow = []
    for i_r in range(len(r)):
        if i_r !=2 and len(r)>8:
            irow.append(r[i_r])
    if len(irow)>8 and "GCA90" not in irow:
        #print(irow)
        sga_players_all_year.append(irow)


save_data_frame_to_csv(sga_players_all_year, file_path_sga_players)